In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
import time 

In [31]:
browser = Browser('chrome')

In [32]:
states = [
    {'abbrev': 'MN', 'name': 'Minnesota'},
    {'abbrev': 'NY', 'name': 'New York'},
    {'abbrev': 'IL', 'name': 'Illinois'},
    {'abbrev': 'IN', 'name': 'Indiana'},
    {'abbrev': 'MI', 'name': 'Michgan'},
    {'abbrev': 'OH', 'name': 'Ohio'},
    {'abbrev': 'PA', 'name': 'Pennslyvania'},
    {'abbrev': 'WI', 'name': 'Wisconsin'}

]

In [34]:
for state in states:
    base_url = 'https://ecos.fws.gov/ecp/report/species-listings-by-state'
    url = f"{base_url}?stateAbbrev={state['abbrev']}&stateName={state['name']}&statusCategory=Listed"
    time.sleep(5)
    browser.visit(url)

    browser.find_by_xpath("//select[@name='species-listings-by-state-report_length']/option[text()='All']").click()

    # Send an HTTP request to the page and retrieve the HTML content
    response = requests.get(url)
    html_content = response.content
    
    # Create a BeautifulSoup object by parsing the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    for row in soup.find_all('tr'):
        columns = row.find_all('td')
        if len(columns) == 5:  
            species_name = columns[0].text.strip()
            common_name = columns[1].text.strip()
            where_listed = columns[2].text.strip()
            region = columns[3].text.strip()
            ESA_listing_status = columns[4].text.strip()

            # Find the link to the species details page
            species_details_link = columns[0].find('a')['href']
            
            # Send an HTTP request to the species details page
            species_details_response = requests.get(species_details_link)
            species_details_soup = BeautifulSoup(species_details_response.content, 'html.parser')
            
            # Extract the image URL from the species details page
            img_tag = species_details_soup.find('img', class_='imageSize')
            image_url = img_tag['src'] if img_tag else None
            
            # Store the extracted data in the data structure
            data.append({
                'State': state['name'],
                'Scientific Name': species_name,
                'Species Name': common_name,
                'Location': where_listed,
                'Region': region,
                'Status': ESA_listing_status,
                'Image URL': image_url
            })
